In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_female_Academy_Award_winners_and_nominees_for_non-gendered_categories"
page = requests.get( url )
soup = BeautifulSoup(page.content, 'html.parser')

#### Raspagem: lista de mulheres nomeadas e ganhadoras do Oscar em categorias em que gêneros competem.

Como alguns elementos ocupam mais que uma linha, não é possível fazer a raspagem iterando entre os múltiplos de cada célula. A solução encontrada foi:

- Para o ano ('year'): tentar converter os 4 primeiros caracteres da célula em inteiro. Se não houver erro, é porque a célula é um ano válido; ele é salvo na variável `new_year`.
- Para o status ('Nominated' ou 'Won'): todas as células de status possuem `<td class="no table-no2" (...)>` para _Nominated_ ou `<td class="yes table-yes2" (...)>` para _Won_. Verifica-se se a célula atual é uma dessas e, se sim, o texto dela, da categoria (título da tabela) e do ano `new_year` é adicionado em suas respectivas listas.


In [3]:
year = []
category = []
status = []
all_tables = soup.find_all(class_='wikitable')

for table in all_tables:
	td = table.find_all('td')
	new_year = 0
	for i in range(0, len(td)):
		try:
			new_year = int(td[i].text[0:4])
		except ValueError:
			pass
		if td[i].get('class'):
			status.append(td[i].text[0:-1])
			category.append(td[0].text[18:-1])
			if new_year != 0:
				year.append(new_year)

oscar_women_df = pd.DataFrame({
	'year': year,
	'category': category,
	'status': status
})

oscar_women_df

,year,category,status
0,2007,Best Animated Feature,Nominated
1,2011,Best Animated Feature,Nominated
2,2012,Best Animated Feature,Won
3,2013,Best Animated Feature,Won
4,2013,Best Animated Feature,Nominated
...,...,...,...
1418,2017,Best Original Screenplay,Nominated
1419,2017,Best Original Screenplay,Nominated
1420,2018,Best Original Screenplay,Nominated
1421,1917,Best Original Screenplay,Nominated


In [4]:
table = all_tables[0]
td = table.find_all('td')
td[14].text

'Won\n'

In [5]:
categoria = soup.find(id="Best_Animated_Feature")
categoria

<span class="mw-headline" id="Best_Animated_Feature">Best Animated Feature</span>